In [3]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

## 1

In [4]:
conn = sqlite3.connect('../data/checking-logs.sqlite')

## 2

In [22]:
commits = pd.io.sql.read_sql(
    """
    SELECT uid, timestamp, numTrials
    FROM checker
    WHERE uid LIKE "user_%" 
          AND status = "ready"
          AND labname = "project1";
    """,
    con=conn,
    parse_dates=['timestamp']
)
commits['timestamp'] = commits['timestamp'].dt.date

commits = commits.groupby(by=['uid', 'timestamp'])['numTrials'].count().unstack()
commits = commits.fillna(0)
commits = commits.cumsum(axis=1)

In [26]:
numOfRows = commits.shape[0]
numOfCols = commits.shape[1]
numOfFrames = numOfCols - 1
dates = [str(date) for date in commits.columns.tolist()]


initial_data = []
for ind, uid in enumerate(commits.index.to_list()):
    y_value = commits.iloc[ind, 0]
    initial_data.append(go.Scatter(x=np.array(dates[0]), y=np.array(y_value), mode='lines', name=uid))

frames = []
for f in range(1, numOfCols):
    x_axis = np.arange(1, f + 1)
    curr_data = []

    for ind, uid in enumerate(commits.index.tolist()):
        y_axis = commits.iloc[ind, :f].values
        curr_data.append(go.Scatter(x=x_axis, y=y_axis, mode='lines', name=uid))
    curr_frame = go.Frame(data=curr_data)
    frames.append(curr_frame)


figure = go.Figure(
    data=initial_data,
     layout = {
        "title":"Dynamic of commits per user in project1",
        "xaxis":{"tickvals": dates, "ticktext": dates, "showline":False},
        "yaxis":{"type":"log", "visible":False, "showline":False},
        "updatemenus":[{"type":"buttons","buttons":[{"method":"animate","label":"play", "args":[None]}]}]
        },
    frames = frames
    )
figure.show()